In [34]:
import pandas as pd
sub_mission=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\공부파일들\카글\house-prices-advanced-regression-techniques\sample_submission.csv")
test=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\공부파일들\카글\house-prices-advanced-regression-techniques\test.csv")
train=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\공부파일들\카글\house-prices-advanced-regression-techniques\train.csv")

In [35]:
import numpy as np

## 교차검증

우선 평가모델부터 만들자

In [36]:
train.drop('Id',inplace=True,axis=1)

In [37]:
num_col=train.select_dtypes(np.number).columns.difference(['SalePrice'])
cate_col=train.select_dtypes("object").columns

In [38]:
import sklearn.impute as skimp
import sklearn.compose as skcom
import sklearn.pipeline as skpip
import sklearn.preprocessing as skpre

In [39]:
# 파이프라인
num_pip=skpip.make_pipeline(skimp.SimpleImputer(strategy="median"),
                            skpre.StandardScaler())
cate_pip=skpip.make_pipeline(skimp.SimpleImputer(strategy="most_frequent"),
                             skpre.OneHotEncoder(handle_unknown="ignore"))

# 합치기
col_pip=skcom.make_column_transformer((num_pip,num_col),
                                      (cate_pip,cate_col))
# 결과
pip_result=col_pip.fit_transform(train)
pre_train=pd.DataFrame(pip_result.toarray(),columns=col_pip.get_feature_names_out())

In [42]:
# 모델
import sklearn.linear_model as sklin
linear_model=sklin.LinearRegression(n_jobs=-1)
linear_model.fit(X=pre_train,y=train["SalePrice"])

LinearRegression(n_jobs=-1)

### 검증해보자
사이킷런의 교차검증은 `비용함수`가 아닌 `효용함수`를 기대하므로,
`MSE`와 같이 비용함수는 음수화해서 `scoring`을 해야한다

In [44]:
import sklearn.model_selection as skmod

In [85]:
lin_mse=-skmod.cross_val_score(linear_model,X=pre_train,y=train["SalePrice"],scoring='neg_root_mean_squared_error',cv=10)

### 결과해석

In [88]:
MSE=pd.DataFrame(lin_mse,columns=["RMSE"])
MSE.describe()

,RMSE
count,1.000000e+01
mean,1.745344e+14
std,1.892555e+14
min,2.296185e+13
25%,4.204109e+13
50%,7.342640e+13
75%,2.732758e+14
max,6.116838e+14


In [87]:
train["SalePrice"].mean()

180921.19589041095

너무 쓰레기임